In [33]:
import tensorflow as tensorflow
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [34]:
# load the trained model, scaler, onehot pickle files
model=load_model('model.h5')

# load the encoder and scalers
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [35]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [36]:
geo_encoded = onehot_encodder_geo.transform(np.array([input_data['Geography']]).reshape(-1,1)).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/Users/aniketkumar/Developer/projects/ann-classification/venv/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [ ]:
# combine one-hot encoded columns with input data
input_data = pd.DataFrame([input_data])

In [38]:
# Encode categorical variables
input_data['Gender']=label_encoder_gender.transform(input_data['Gender'])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [39]:
input_data = pd.concat([input_data.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [40]:
# Scaling the input data
input_scaled = scaler.transform(input_data)

In [42]:
# Predict churn
prediction=model.predict(input_scaled) 
prediction

1/1 [==============================] - 0s 14ms/step


array([[0.04223176]], dtype=float32)

In [43]:
prediction_proba = prediction[0][0]
prediction_proba

0.04223176

In [44]:
if prediction_proba > 0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
